In [ ]:
# !apt-get update
# !apt-get install g++ openjdk-8-jdk 
# !pip3 install konlpy JPype1-py3
# !bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash
from konlpy.tag import Mecab

mecab = Mecab()
mecab.morphs('동해물과 백두산이 마르고 닳도록')

In [ ]:
!pip install transformers
!pip install tensorboardx

In [ ]:
# !pip install pyrouge --upgrade
# !pip install https://github.com/bheinzerling/pyrouge/archive/master.zip
# !pip install pyrouge
# !pip show pyrouge
# !git clone https://github.com/andersjo/pyrouge.git
from pyrouge import Rouge155
# !pyrouge_set_rouge_path 'pyrouge/tools/ROUGE-1.5.5'

In [ ]:
!sudo apt-get update -y

In [ ]:
!sudo apt-get install -y libxml-parser-perl

In [ ]:
# # %%shell
# !cd pyrouge/tools/ROUGE-1.5.5/data
# !rm WordNet-2.0.exc.db # only if exist
# !cd WordNet-2.0-Exceptions
# !rm WordNet-2.0.exc.db # only if exist

# !./buildExeptionDB.pl . exc WordNet-2.0.exc.db
# !cd ../
# !ln -s WordNet-2.0-Exceptions/WordNet-2.0.exc.db WordNet-2.0.exc.db

In [ ]:
# %cd /content
# !git clone https://github.com/HaloKim/KorBertSum.git
# %cd /content/KorBertSum

In [ ]:
# pip install torch torchvision

In [ ]:
from transformers import BertTokenizer, BertModel, BertConfig

config = BertConfig.from_pretrained('bert-base-multilingual-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")

In [ ]:
import pandas as pd
import json

train = pd.read_csv('json_to_csv_train_신문기사.csv').drop('Unnamed: 0', axis=1)
print(len(train))
train.head()

In [ ]:
valid = pd.read_csv('json_to_csv_valid_신문기사.csv').drop('Unnamed: 0', axis=1)
print(len(valid))
valid.head()

In [ ]:
all_data = pd.concat([train,valid])
print(len(all_data))
all_data.head()

In [ ]:
from sklearn.model_selection import train_test_split

train_set = train
valid_set, test_set = train_test_split(valid, test_size = 0.1)

# from sklearn.model_selection import train_test_split

# train_set, test_set = train_test_split(all_data, test_size = 0.4)
# valid_set, test_set = train_test_split(test_set, test_size = 0.5)

In [ ]:
print(len(train_set))
print(len(valid_set))
print(len(test_set))

In [ ]:
import ast
from tqdm import tqdm
import os

list_dic = []
for idx, row in train_set.iterrows():
  raw = row['article_orginal']
  target_idx = ast.literal_eval(row['extractive'])

  sentences = raw.split(',')
  src = [i.split(' ') for i in sentences]
  tgt = [a for i,a in enumerate(src) if i in target_idx]
  
  mydict = {}
  mydict['src'] = src
  mydict['tgt'] = tgt
  list_dic.append(mydict)
        
temp = []
for i,a in enumerate(tqdm(list_dic)):
  if (i+1)%1000!=0:
      temp.append(a)
  else:
      filename = 'korean.'+'train'+'.'+str(i//1000)+'.json'
      with open('KorBertSum/json/'+filename, "w", encoding='utf-8') as json_file:
          json.dump(temp, json_file, ensure_ascii=False)
      temp = []

In [ ]:
try:
  os.mkdir('KorBertSum/json/val')
except:
  pass

list_dic = []
for idx, row in valid_set.iterrows():
        raw = row['article_orginal']
        target_idx = ast.literal_eval(row['extractive'])

        sentences = raw.split(',')
        src = [i.split(' ') for i in sentences]
        tgt = [a for i,a in enumerate(src) if i in target_idx]
        
        mydict = {}
        mydict['src'] = src
        mydict['tgt'] = tgt
        list_dic.append(mydict)
        
temp = []
for i,a in enumerate(tqdm(list_dic)):
    
    if (i+1)%1000!=0:
        temp.append(a)
    else:
        filename = 'korean.'+'valid'+'.'+str(i//1000)+'.json'
        with open('KorBertSum/json/val/'+filename, "w", encoding='utf-8') as json_file:
            json.dump(temp, json_file, ensure_ascii=False)
        temp = []

In [ ]:
try:
  os.mkdir('KorBertSum/json/test')
except:
  pass

list_dic = []
for idx, row in test_set.iterrows():
        raw = row['article_orginal']
        target_idx = ast.literal_eval(row['extractive'])

        sentences = raw.split(',')
        src = [i.split(' ') for i in sentences]
        tgt = [a for i,a in enumerate(src) if i in target_idx]
        
        mydict = {}
        mydict['src'] = src
        mydict['tgt'] = tgt
        list_dic.append(mydict)
        
temp = []
for i,a in enumerate(tqdm(list_dic)):
    
    if (i+1)%1000!=0:
        temp.append(a)
    else:
        filename = 'korean.'+'test'+'.'+str(i//1000)+'.json'
        with open('KorBertSum/json/test/'+filename, "w", encoding='utf-8') as json_file:
            json.dump(temp, json_file, ensure_ascii=False)
        temp = []

In [ ]:
# 데이터 처리모드, json경로, 저장경로, 로그파일
!python KorBertSum/src/preprocess.py \
  -mode format_to_bert \
  -raw_path KorBertSum/json \
  -save_path KorBertSum/bert_data \
  -dataset train

In [ ]:
try:
  os.mkdir('KorBertSum/bert_data/test')
except:
  pass

!python KorBertSum/src/preprocess.py \
  -mode format_to_bert \
  -raw_path KorBertSum/json/val \
  -save_path KorBertSum/bert_data/test \
  -dataset valid

In [ ]:
try:
  os.mkdir('KorBertSum/bert_data/test')
except:
  pass

!python KorBertSum/src/preprocess.py \
  -mode format_to_bert \
  -raw_path KorBertSum/json/test \
  -save_path KorBertSum/bert_data/test \
  -dataset test

In [ ]:
!python KorBertSum/src/train.py \
  -mode train \
  -encoder transformer \
  -bert_data_path KorBertSum/bert_data/korean \
  -model_path KorBertSum/models/bert_classifier_3 \
  -log_file KorBertSum/logs/train_3.txt \
  -lr 0.002 -visible_gpus 0 -gpu_ranks 0 -world_size 1 -report_every 1000 -dropout 0.5 \
  -save_checkpoint_steps 5000 -batch_size 3000 -decay_method noam -train_steps 65000 -accum_count 1 \
  -optim adamW

In [ ]:
!python KorBertSum/src/train.py \
  -mode validate \
  -bert_data_path KorBertSum/bert_data/test/korean \
  -model_path KorBertSum/models/bert_trans_2 \
  -visible_gpus 0 -gpu_ranks 0 -batch_size 3000 \
  -log_file KorBertSum/logs/val_trans_2.txt \
  -temp_dir KorBertSum/temp \
  -result_path KorBertSum/results \
  -lr 0.002 \
#   -test_all 1 \
#   -test_from KorBertSum/models/bert_classifier/model_step_30000.pt \

In [ ]:
!python KorBertSum/src/train.py \
  -mode test \
  -bert_data_path KorBertSum/bert_data/test/korean \
  -visible_gpus 0 -gpu_ranks 0 -batch_size 3000 \
  -log_file KorBertSum/logs/test.txt \
  -temp_dir KorBertSum/temp \
  -result_path KorBertSum/results \
  -test_from KorBertSum/models/bert_classifier_3/model_step_65000.pt